In [6]:
import pandas as pd
import os

sym_dir = 'Symbols'
sym_files = [os.path.join(sym_dir, f) for f in os.listdir(sym_dir)]

In [578]:
class Property:
    idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'value': 7}
    just_dict = {1: 'Upper Left', 2: 'Middle Left', 3: 'Lower Left', 4: 'Upper Center', 5: 'Middle Center', 6: 'Lower Center', 7: 'Upper Right', 8: 'Middle Right', 9: 'Lower Right'}
    rotation_dict = {0: '0', 1: '90', 2: '180', 3: '270'}
    vis_dict = {0: 'Hidden', 2: 'Hidden-wProperty', 3: 'Visible', 4: 'Visible-wProperty'}
    
    def __init__(self, line_str, identifier):
        vals = line_str.split()
        
        self.x = int(vals[self.idx['x']])
        self.y = int(vals[self.idx['y']])
        self.size = int(vals[self.idx['size']])
        self.rotation = self.rotation_dict[int(vals[self.idx['rotation']])]
        self.justification = self.just_dict[int(vals[self.idx['justification']])]
        self.visible = self.vis_dict[int(vals[self.idx['visible']])]
        self.value = ' '.join(vals[self.idx['value']:])[len(identifier):] # Account for spaces in property value
        self.property = identifier.split('=')[0]

In [579]:
class PinType(Property):
    def __init__(self, line_str):
        super().__init__(line_str, 'PINTYPE=')
        
class PinNumber(Property):
    def __init__(self, line_str):
        super().__init__(line_str, '#=')

In [580]:
class PinName(Property):
    idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 7, 'value': 9} # Line row values to string index
    vis_dict = {0: 'Hidden', 1: 'Visible'}
    
    def __init__(self, line_str):
        super().__init__(line_str, '')

In [608]:
class Box:
    idx = {'x1': 1, 'y1': 2, 'x2': 3, 'y2': 4}
    def __init__(self, line_str):
        vals = line_str.split()
        self.x1 = vals[self.idx['x1']]
        self.x2 = vals[self.idx['x2']]
        self.y1 = vals[self.idx['y1']]
        self.y2 = vals[self.idx['y2']]

In [581]:
class Pin:
    idx = {'id': 1, 'x1': 2, 'y1': 3, 'x2': 4, 'y2': 5, 'side': 7, 'inverted': 8} # Pin row to string index
    side_dict = {0: 'Top', 1: 'Bottom', 2: 'Left', 3: 'Right'}
    
    def __init__(self, line_str):
        vals = line_str.split()
        
        self.pid = vals[self.idx['id']]
        self.set_line_pos(vals[self.idx['x1']], vals[self.idx['x2']], vals[self.idx['y1']], vals[self.idx['y2']])
        self.side = self.side_dict[int(vals[self.idx['side']])]
        self.inverted = bool(int(vals[self.idx['inverted']]))
    
    # number: PinNumber Object
    def set_number(self, number):
        self.Number = number
    
    # ptype: PinType Object
    def set_type(self, ptype):
        self.Type = ptype
        
    def set_line_pos(self, x1, x2, y1, y2):
        self.line_pos = (int(x1), int(x2), int(y1), int(y2))
        
    # name: PinName Object
    def set_name(self, name):
        self.Name = name

In [582]:
class SymProperty(Property):
    def __init__(self, line_str):
        identifier = line_str.split()[self.idx['value']].split('=')[0] + '='
        super().__init__(line_str, identifier)

In [621]:
class Symbol:
    pins = {}
    boxes = []
    sym_headers = ['K','|R','Y','U','b']
    k_idx = {'name': 2}
    d_idx = {'date': 1} # Date row indexes
    u_idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'value': 7}
    y_idx = {'value': 1}
    vis_dict = {0: 'Hidden', 2: 'Hidden-wProperty', 3: 'Visible', 4: 'Visible-wProperty'}
    sym_type = {0: 'Composite', 1: 'Module', 2: 'Pin', 4: 'Annotate', 5: 'Border'}
    
    def __init__(self):
        pass
        
    def add_pin(self, pin):
        self.pins[pin.Number.value] = pin
        
    def units_to_mils(unit):
        return 100*unit/254000
    
    def parse_sym(self, line_str):
        vals = line_str.split()
        if vals[0] == 'K':
            self.name = vals[self.k_idx['name']]
        elif vals[0] == '|R':
            self.save_date = vals[self.d_idx['date']]
        elif vals[0] == 'Y':
            self.symbol_type = self.sym_type[int(vals[self.y_idx['value']])]
        elif vals[0] == 'U':
            self.parse_property(line_str)
        elif vals[0] == 'b':
            self.boxes += [Box(line_str)]
            
    def parse_property(self, line_str):
        sym_property = SymProperty(line_str)
        if sym_property.property == 'DEVICE':
            self.device = sym_property
        elif sym_property.property == 'FORWARD_PCB':
            self.forward = sym_property
        elif sym_property.property == 'HETERO':
            self.hetero = sym_property
        elif sym_property.property == 'PKG_STYLE':
            self.pkg_style = sym_property
        elif sym_property.property == 'PKG_TYPE':
            self.pkg_type = sym_property
        elif sym_property.property == 'PLACE':
            self.place = sym_property
        elif sym_property.property == 'REFDES':
            self.refdes = sym_property
        elif sym_property.property == 'VALUE':
            self.value = sym_property
        elif sym_property.property == 'NAME_PLACEHOLDER':
            self.name_placeholder = sym_property
        
    
    def parse_pin(self, pin_str_list):
        for line_str in pin_str_list:
            if line_str.startswith('P '):
                p = Pin(line_str)
            elif line_str.startswith('L '):
                p.set_name(PinName(line_str))
            elif '#=' in line_str:
                p.set_number(PinNumber(line_str))
            elif 'PINTYPE=' in line_str:
                p.set_type(PinType(line_str))
                
        self.add_pin(p)
        return p

In [622]:
def get_symbol(file_path):
    sym = Symbol()
    NEWPIN = False
    prev_header = None
    pin_cnt = 0
    pin_list = []
    pin_str_list = []

    with open(file_path, 'r') as sym_file:
        for l in sym_file:
            if l.split()[0] in sym.sym_headers:
                sym.parse_sym(l)
            elif l.startswith('P '):
                if pin_cnt > 0:
                    pin_list += [sym.parse_pin(pin_str_list)]
                NEWPIN = True
                pin_str_list = [l] # Reset List on each new pin
                pin_cnt += 1
            elif NEWPIN:
                pin_str_list += [l]
                if pin_cnt > 0:
                    pass
                
            if l.startswith('|GRPHSTL'):
                pass # TODO: add graphic style and font to previous properties
            prev_header = l.split()[0]
    return sym

In [623]:
sym = get_symbol(sym_files[0])

In [619]:
# for p in pin_list:
#     print('Name: {}, ID: {}, Number: {}, Type: {}, Side: {}, Inv: {}'.format(p.Name.value, p.pid, p.Number.value, p.Type.value, p.side, p.inverted))
#     #print('Name: {}, Origin: {}, Side: {}, Rotation: {}'.format(p.name, p.value.txt_just, p.side, p.value.rotation))

In [620]:
sym.save_date

'16:44:41_11-4-19'

In [624]:
sym.symbol_type

'Composite'